In [ ]:
%pip install slack-sdk

In [ ]:
# Cell 1 — Imports & Load status.json
import os
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

# Environment variables
SLACK_TOKEN   = os.environ["SLACK_BOT_TOKEN"]
SLACK_CHANNEL = os.environ.get("SLACK_CHANNEL", "#ocr-reports")
STATUS_PATH   = "status.json"

# Initialize Slack client
slack = WebClient(token=SLACK_TOKEN)

# Load the status file (dict with keys: repository, commit, processed, errors)
with open(STATUS_PATH, "r", encoding="utf-8") as f:
    status = json.load(f)

repo      = status.get("repository", "<unknown>")
commit    = status.get("commit",     "<unknown>")
processed = status.get("processed",  [])
errors    = status.get("errors",     [])
topic_id = os.environ.get("topic_id")

In [ ]:
# Cell 2 — Build & send report (with processed files list)
count_processed = len(processed)
count_errors    = len(errors)

text = (
    "*📊 ETL & TWYD Report*\n"
    f"• Repository: `{repo}`\n"
    f"• Commit:     `{commit}`\n"
    f"• Processed:  *{count_processed}* files\n"
    f"• Topic id:     *{topic_id}* \n"
)

# List processed files
if count_processed > 0:
    text += "\n\n*Processed files:*\n"
    # List up to 10 filenames
    for fn in processed[:10]:
        text += f"  – `{fn}`\n"
    if count_processed > 10:
        text += f"  _…and {count_processed - 10} more_\n"

# List errors if any
if count_errors > 0:
    text += "\n\n*Errors:*\n"
    for err in errors[:10]:
        text += f"  – `{err['file']}`: {err['error']}\n"
    if count_errors > 10:
        text += f"  _…and {count_errors - 10} more_\n"

try:
    resp = slack.chat_postMessage(channel=SLACK_CHANNEL, text=text)
    print(f"✅ Report sent to {SLACK_CHANNEL} (ts={resp['ts']})")
except SlackApiError as e:
    print(f"❌ Slack API error: {e.response['error']}")